# Multi-Agent Conversation and Stand-up Comedy

## Setup

In [1]:
from utils import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()
llm_config = {"model": "gpt-3.5-turbo"}

## Define an AutoGen agent

In [2]:
from autogen import ConversableAgent

agent = ConversableAgent(
name = 'chatbot',
    llm_config = llm_config,
    human_input_mode = 'NEVER',
)

In [3]:
reply = agent.generate_reply(
messages = [{'content' : 'Tell me a joke', 'role' : 'user'}]
)
print(reply)

Sure! Here's a joke for you:

Why couldn't the bicycle stand up by itself?

Because it was two tired!


In [4]:
reply = agent.generate_reply(
messages = [{'content' : 'Repeat the joke.', 'role': 'user'}]
)
print(reply)

Of course! Just let me know which joke you'd like me to repeat for you.


## Conversation

Setting up a conversation between two agents, Cathy and Joe, where the memory of their interactions is retained.

In [7]:
cathy = ConversableAgent(
name = 'Cathy',
system_message = 'Your name is Cathy and you are a stand-up comedian.',
llm_config = llm_config,
human_input_mode = 'NEVER',
)

joe = ConversableAgent(
name = 'joe',
system_message = 'Your name is Joe and you are a stand-up comedian.'
                 'Start the next joke from the punchline of the previous joke.',
llm_config = llm_config,
human_input_mode = 'NEVER',
)

In [10]:
chat_result = joe.initiate_chat(
    recipient=cathy, 
    message="I'm Joe. Cathy, let's keep the jokes rolling.",
    max_turns=2,
)

joe (to Cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
Cathy (to joe):

Absolutely, Joe! Let's kick things off with a classic. Why did the scarecrow win an award? Because he was outstanding in his field!

--------------------------------------------------------------------------------
joe (to Cathy):

Well, Cathy, that scarecrow must've really known how to "branch" out in his career!

--------------------------------------------------------------------------------
Cathy (to joe):

Haha, good one, Joe! Sounds like that scarecrow wasn't just strawng, but also had a great sense of humor!

--------------------------------------------------------------------------------


## Print some results

You can print out:

1. Chat history
2. Cost
3. Summary of the conversation

In [11]:
import pprint

pprint.pprint(chat_result.chat_history)

[{'content': "I'm Joe. Cathy, let's keep the jokes rolling.",
  'role': 'assistant'},
 {'content': "Absolutely, Joe! Let's kick things off with a classic. Why did "
             'the scarecrow win an award? Because he was outstanding in his '
             'field!',
  'role': 'user'},
 {'content': "Well, Cathy, that scarecrow must've really known how to "
             '"branch" out in his career!',
  'role': 'assistant'},
 {'content': "Haha, good one, Joe! Sounds like that scarecrow wasn't just "
             'strawng, but also had a great sense of humor!',
  'role': 'user'}]


In [12]:
pprint.pprint(chat_result.cost)

{'usage_excluding_cached_inference': {'gpt-3.5-turbo-0125': {'completion_tokens': 82,
                                                             'cost': 0.0002295,
                                                             'prompt_tokens': 213,
                                                             'total_tokens': 295},
                                      'total_cost': 0.0002295},
 'usage_including_cached_inference': {'gpt-3.5-turbo-0125': {'completion_tokens': 82,
                                                             'cost': 0.0002295,
                                                             'prompt_tokens': 213,
                                                             'total_tokens': 295},
                                      'total_cost': 0.0002295}}


In [13]:
pprint.pprint(chat_result.summary)

("Haha, good one, Joe! Sounds like that scarecrow wasn't just strawng, but "
 'also had a great sense of humor!')


## Get a better summary of the conversation

In [14]:
chat_result = joe.initiate_chat(
    cathy, 
    message="I'm Joe. Cathy, let's keep the jokes rolling.", 
    max_turns=2, 
    summary_method="reflection_with_llm",
    summary_prompt="Summarize the conversation",
)

joe (to Cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
Cathy (to joe):

Absolutely, Joe! Let's kick things off with a classic. Why did the scarecrow win an award? Because he was outstanding in his field!

--------------------------------------------------------------------------------
joe (to Cathy):

Well, Cathy, that scarecrow must've really known how to "branch" out in his career!

--------------------------------------------------------------------------------
Cathy (to joe):

Haha, good one, Joe! Sounds like that scarecrow wasn't just strawng, but also had a great sense of humor!

--------------------------------------------------------------------------------


In [15]:
pprint.pprint(chat_result.summary)

('The conversation highlights a playful exchange of jokes between Joe and '
 'Cathy about a talented scarecrow winning an award.')


## Chat Termination

Chat can be terminated using a termination conditions.

In [30]:
cathy = ConversableAgent(
    name="cathy",
    system_message=
    "Your name is Cathy and you are a stand-up comedian. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"],
)

joe = ConversableAgent(
    name="joe",
    system_message=
    "Your name is Joe and you are a stand-up comedian. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"] or "Goodbye" in msg["content"],
)

In [32]:
chat_result = joe.initiate_chat(
    recipient=cathy,
    message="I'm Joe. Cathy, let's keep the jokes rolling."
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Hey Joe! Why did the scarecrow win an award? Because he was outstanding in his field!

--------------------------------------------------------------------------------
joe (to cathy):

Haha, that's a good one, Cathy! Scarecrows getting recognition for their hard work, love it!

--------------------------------------------------------------------------------
cathy (to joe):

Thanks, Joe! Gotta give credit where credit is due, right? So, why did the tomato turn red? Because it saw the salad dressing!

--------------------------------------------------------------------------------
joe (to cathy):

Haha, that's a classic! The tomato couldn't help but blush when it saw what was coming its way. Love it, Cathy!

--------------------------------------------------------------------------------
cathy (to joe):

Glad you enjoyed it, Joe

In [ ]:
cathy.send(message="What's last joke we talked about?", recipient=joe)

cathy (to joe):

What's last joke we talked about?

--------------------------------------------------------------------------------
joe (to cathy):

The last joke was about the tomato turning red because it saw the salad dressing! That was a good one, Cathy.

--------------------------------------------------------------------------------
